In [ ]:
from pyspark.sql import SparkSession

# Criar uma sessão Spark
spark = SparkSession.builder\
    .appName("ApiDataExtraction")\
    .config("spark.master", "local[*]")\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/09 16:49:21 WARN Utils: Your hostname, ysla, resolves to a loopback address: 127.0.1.1; using 192.168.15.141 instead (on interface wlo1)
25/08/09 16:49:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/09 16:49:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!pip install sodapy

  Using cached sodapy-2.2.0-py2.py3-none-any.whl.metadata (15 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
Using cached sodapy-2.2.0-py2.py3-none-any.whl (15 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [sodapy]2m2/6 [charset_normalizer]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
from sodapy import Socrata

# URL da API
url = 'https://data.cityofchicago.org/api/v3/views/85ca-t3if/query.json'

client = Socrata("data.cityofchicago.org", None)
results = client.get("85ca-t3if", limit=100000)


In [7]:
results[0]

{'crash_record_id': '01592f02a8ea88406703e6c88b244e37d7163905ae1a76420eaa4ecd71ca4f2333d52de953f73555f250691b38b0e9e67e924d2a19480e13a375af34beb80e60',
 'crash_date_est_i': 'Y',
 'crash_date': '2025-08-09T01:30:00.000',
 'posted_speed_limit': '30',
 'traffic_control_device': 'NO CONTROLS',
 'device_condition': 'NO CONTROLS',
 'weather_condition': 'CLEAR',
 'lighting_condition': 'DARKNESS, LIGHTED ROAD',
 'first_crash_type': 'PARKED MOTOR VEHICLE',
 'trafficway_type': 'ONE-WAY',
 'alignment': 'STRAIGHT AND LEVEL',
 'roadway_surface_cond': 'DRY',
 'road_defect': 'NO DEFECTS',
 'report_type': 'NOT ON SCENE (DESK REPORT)',
 'crash_type': 'NO INJURY / DRIVE AWAY',
 'hit_and_run_i': 'Y',
 'damage': 'OVER $1,500',
 'date_police_notified': '2025-08-09T01:50:00.000',
 'prim_contributory_cause': 'UNABLE TO DETERMINE',
 'sec_contributory_cause': 'NOT APPLICABLE',
 'street_no': '4820',
 'street_direction': 'W',
 'street_name': 'RACE AVE',
 'beat_of_occurrence': '1532',
 'num_units': '2',
 'most_se

In [8]:
from pyspark.sql.types import ArrayType, StructType, StructField, DoubleType, IntegerType, StringType, TimestampType

# Inferir o schema para o Dataframe
# Definir o schema manualmente para garantir consistência
schema = StructType([
    StructField("crash_record_id", StringType(), True), 
    StructField("crash_date", StringType(), True), 
    StructField("posted_speed_limit", StringType(), True),
    StructField("traffic_control_device", StringType(), True),
    StructField("device_condition", StringType(), True),
    StructField("weather_condition", StringType(), True),
    StructField("lighting_condition", StringType(), True), 
    StructField("first_crash_type", StringType(), True),
    StructField("trafficway_type", StringType(), True),
    StructField("alignment", StringType(), True),
    StructField("roadway_surface_cond", StringType(), True),
    StructField("road_defect", StringType(), True),
    StructField("report_type", StringType(), True),
    StructField("crash_type", StringType(), True),
    StructField("hit_and_run_i", StringType(), True),
    StructField("damage", StringType(), True), 
    StructField("date_police_notified", StringType(), True),
    StructField("prim_contributory_cause", StringType(), True),
    StructField("sec_contributory_cause", StringType(), True),
    StructField("street_no", StringType(), True),
    StructField("street_direction", StringType(), True),
    StructField("street_name", StringType(), True),
    StructField("beat_of_occurrence", StringType(), True),
    StructField("num_units", StringType(), True),
    StructField("most_severe_injury", StringType(), True),
    StructField("injuries_total", StringType(), True),
    StructField("injuries_fatal", StringType(), True),
    StructField("injuries_incapacitating", StringType(), True),
    StructField("injuries_non_incapacitating", StringType(), True),
    StructField("injuries_reported_not_evident", StringType(), True),
    StructField("injuries_no_indication", StringType(), True),
    StructField("injuries_unknown", StringType(), True),
    StructField("crash_hour", StringType(), True),
    StructField("crash_day_of_week", StringType(), True),
    StructField("crash_month", StringType(), True),
    StructField("latitude", StringType(), True),
    StructField("longitude", StringType(), True),
    StructField("location", StructType([
        StructField("type", StringType(), True),
        StructField("coordinates", ArrayType(StringType()), True)
    ]), True),
    StructField(":@computed_region_rpca_8um6", StringType(), True)
])

# Criar um Dataframe a partir da lista de dicionarios (dados da API)

df = spark.createDataFrame(results, schema=schema)

df.printSchema()

root
 |-- crash_record_id: string (nullable = true)
 |-- crash_date: string (nullable = true)
 |-- posted_speed_limit: string (nullable = true)
 |-- traffic_control_device: string (nullable = true)
 |-- device_condition: string (nullable = true)
 |-- weather_condition: string (nullable = true)
 |-- lighting_condition: string (nullable = true)
 |-- first_crash_type: string (nullable = true)
 |-- trafficway_type: string (nullable = true)
 |-- alignment: string (nullable = true)
 |-- roadway_surface_cond: string (nullable = true)
 |-- road_defect: string (nullable = true)
 |-- report_type: string (nullable = true)
 |-- crash_type: string (nullable = true)
 |-- hit_and_run_i: string (nullable = true)
 |-- damage: string (nullable = true)
 |-- date_police_notified: string (nullable = true)
 |-- prim_contributory_cause: string (nullable = true)
 |-- sec_contributory_cause: string (nullable = true)
 |-- street_no: string (nullable = true)
 |-- street_direction: string (nullable = true)
 |-- s

In [9]:
df.show(truncate=False, vertical=True, n=1)

25/08/09 16:52:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/08/09 16:52:22 WARN TaskSetManager: Stage 0 contains a task of very large size (13915 KiB). The maximum recommended task size is 1000 KiB.
Exception ignored in: <_io.BufferedWriter name=5>                   (0 + 1) / 1]
Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 200, in manager
BrokenPipeError: [Errno 32] Broken pipe


-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------
 crash_record_id               | 01592f02a8ea88406703e6c88b244e37d7163905ae1a76420eaa4ecd71ca4f2333d52de953f73555f250691b38b0e9e67e924d2a19480e13a375af34beb80e60 
 crash_date                    | 2025-08-09T01:30:00.000                                                                                                          
 posted_speed_limit            | 30                                                                                                                               
 traffic_control_device        | NO CONTROLS                                                                                                                      
 device_condition              | NO CONTROLS                                                                                                                      
 weather_condition    

In [10]:
df.write.mode("overwrite").parquet("Traffic_Crashes")

25/08/09 17:09:17 WARN TaskSetManager: Stage 1 contains a task of very large size (13915 KiB). The maximum recommended task size is 1000 KiB.
